In [1]:
import numpy as np
import pandas as pd
import MeCab
import json
import re
from collections import Counter

/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [141]:
# 名詞・複合名詞のリストを返す
def get_noun_list(doc, join=True):
    mecab_param = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
    mecab_param.parse("")
    node = mecab_param.parseToNode(doc)
    
    noun_list = []
    noun = []
    while node:
        if len(node.surface) == 0:
            node = node.next
            continue

        hinshi = node.feature.split(',')
        if hinshi[0] in ['名詞', '接頭詞']:
            if join:
                noun.append(node.surface)
            else:
                noun_list.append(node.surface)
        elif (len(noun) > 0) and join:            
            noun_list.append(''.join(noun))
            noun = []
        
        node = node.next
    
    if (len(noun) > 0) and join:
        noun_list.append(''.join(noun))

    return noun_list

In [4]:
def read_jasonl(filename):
    with open(filename) as f:
        return [json.loads(line.rstrip('\r\n')) for line in f.readlines()]

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist if len(item) is not 0]

In [5]:
wiki_data = read_jasonl("../data/jawiki-cirrussearch-dump_of_Compound.jsonl")

In [7]:
with open("../data/compound_train.json") as f:
    train = json.load(f)['entry']
    
train_char_dict = dict([[str(entry['WikipediaID']), entry['Attributes']['特性']] for entry in train])

In [140]:
train_wiki_doc_df = \
pd.DataFrame({
    "_id": [entry['index']['_id'] for entry in wiki_data]
    , "document": [entry['opening_text'] if ('opening_text' in entry.keys() and entry['opening_text'] != None) else entry['text'] for entry in wiki_data]
    , "auxiliary_text": [' '.join(text['auxiliary_text']) for text in wiki_data]
}).pipe(
    lambda x: x.loc[x._id.isin(train_char_dict.keys())]
).reset_index(drop=True)

In [97]:
Counter(flatten(train_char_dict.values())).most_common()

[('毒性', 51),
 ('不燃性', 36),
 ('刺激性', 29),
 ('可燃性', 23),
 ('無色の液体', 23),
 ('有毒', 20),
 ('腐食性', 20),
 ('無色液体', 19),
 ('有害', 19),
 ('劇物', 18),
 ('不溶', 18),
 ('揮発性', 17),
 ('吸湿性', 17),
 ('潮解性', 16),
 ('発癌性', 15),
 ('無色', 14),
 ('可溶', 14),
 ('無臭', 13),
 ('固体', 13),
 ('水溶性', 12),
 ('塩基性', 12),
 ('無色結晶', 12),
 ('白色結晶', 12),
 ('引火性', 11),
 ('発がん性', 10),
 ('無色固体', 10),
 ('白色固体', 9),
 ('急性毒性', 7),
 ('液体', 7),
 ('無色気体', 7),
 ('猛毒', 7),
 ('白色粉末', 7),
 ('無色の気体', 7),
 ('反磁性', 6),
 ('環境への危険性', 6),
 ('白色の固体', 6),
 ('気体', 5),
 ('無色の固体', 5),
 ('粉末', 5),
 ('爆発性', 5),
 ('無色透明の液体', 5),
 ('変異原性', 5),
 ('アルカリ性', 5),
 ('不溶性', 5),
 ('刺激臭', 4),
 ('酸性', 4),
 ('Flammable', 4),
 ('強い可燃性', 4),
 ('混和性', 4),
 ('無色透明', 4),
 ('酸化性', 4),
 ('有害性', 4),
 ('微溶', 4),
 ('難溶', 3),
 ('求核性', 3),
 ('アミン臭', 3),
 ('無色または淡黄色液体', 3),
 ('依存性', 3),
 ('抗腫瘍活性', 3),
 ('黒色固体', 3),
 ('毒物', 3),
 ('水に易溶', 3),
 ('加水分解', 3),
 ('耐熱性', 3),
 ('揮発性の液体', 3),
 ('無色透明液体', 3),
 ('不活性', 3),
 ('黄色固体', 3),
 ('不安定', 3),
 ('白色の結晶', 3),
 ('特異臭', 3),
 ('無色の結晶'

In [186]:
[text['auxiliary_text'] for text in wiki_data if text['auxiliary_text'] and ]

['この項目は、化学に関連した書きかけの項目です。この項目を加筆・訂正などしてくださる協力者を求めています（プロジェクト:化学／Portal:化学）。',
 'ハロン名 フロン名 体系名 その他の名 化学式 分子量 特定 用途 ハロン11 フロン11 フルオロトリクロロメタン  CCl3F 137.368 特定フロン  ハロン14 フロン14 テトラフルオロメタン 四フッ化炭素 CF4 088.004   ハロン104 フロン10 テトラクロロメタン 四塩化炭素 CCl4 153.82   ハロン112 フロン21 ジクロロフルオロメタン  CHCl2F 102.92   ハロン122 フロン12 ジクロロジフルオロメタン  CCl2F2 120.91 特定フロン  ハロン242 フロン114 ジクロロテトラフルオロエタン  CClF2-CClF2 170.92 特定フロン  ハロン1001  ブロモメタン 臭化メチル CH3Br 094.939   ハロン1011  ブロモクロロメタン  CH2ClBr 129.384   ハロン1103  トリブロモフルオロメタン  CBr3F 270.72   ハロン1201  ブロモジフルオロメタン  CHBrF2 130.92   ハロン1202  ジブロモジフルオロメタン  CBr2F2 209.815   ハロン1211  ブロモクロロジフルオロメタン  CClBrF2 165.364 特定ハロン 消火剤 ハロン1301  ブロモトリフルオロメタン  CBrF3 148.909 特定ハロン 消火剤 ハロン2001  ブロモエタン 臭化エチル CBrH2-CH3 108.966   ハロン2302  ジブロモトリフルオロエタン  CBrF2-CHBrF 241.832   ハロン2312  クロロジブロモトリフルオロエタン  CClBr2-CF3 276.277   ハロン2402  ジブロモテトラフルオロエタン  CBrF2-CBrF2 259.822 特定ハロン 消火剤 ハロン2600 フロン116 ヘキサフルオロエタン 六フッ化二炭素 CF3-CF3 138.01   ハロン10001  ヨードメタン ヨウ化メチル CH3I 141.939',
 'ラウレス硫酸ナトリウム  分子式 CH3(CH2)10CH2(OC

In [178]:
def extract_characteristic(noun_list):
    result_char = [noun for noun in noun_list \
                   if (is_characteristic(noun) or is_color(noun) or is_status(noun) or is_smell(noun) or is_taste(noun))]
    
    return list(set(result_char))

def is_characteristic(word):
    patt = r'[^特|危険|男|女]+性$'
    return re.match(patt, word)

def is_color(word):
    patt = r'[^着|染]+色$'
    return re.match(patt, word)

def is_status(word):
    patt = r'(気体|液体|個体|結晶|粉末)$'
    return re.match(patt, word)

def is_smell(word):
    patt = r'\w+臭$'
    return re.match(patt, word)

def is_taste(word):
    patt = r'[^意]+味$'
    return re.match(patt, word)

In [179]:
train_wiki_doc_df["extracted_char"] = \
train_wiki_doc_df.apply(
    lambda x: extract_characteristic(get_noun_list(x.document)) + extract_characteristic(get_noun_list(x.auxiliary_text, join=False))
    , axis=1
)

In [180]:
is_contain_list1 = []
is_contain_list2 = []
predict_df = pd.DataFrame()

for _id, group in train_wiki_doc_df.groupby('_id'):
    if (not train_char_dict.get(_id)) or (len(group.extracted_char.values[0]) == 0):
        continue
    
    tmp_df = pd.DataFrame({"_id": _id, "extracted_char": group.extracted_char.values[0]})
    true_char_list = train_char_dict[_id]
    
    # contains extraction data in train data
    char_str = ','.join(true_char_list)
    result1 = tmp_df.apply(lambda x: True if re.search(x.extracted_char, char_str) else False, axis=1).values.tolist()
    is_contain_list1 += result1   

    # contains train data in extraction data
    r2 = '|'.join(true_char_list).replace('(', '\(').replace(')', '\)')
    result2 = tmp_df.extracted_char.str.contains(r2).values.tolist()
    is_contain_list2 += result2
    
    predict_df = predict_df.append(tmp_df.assign(label = np.array(result1) + np.array(result2)))

print("Num of True:", len(flatten(train_char_dict.values())))
print("Extracted:", len(predict_df))
print(predict_df.label.mean())

Num of True: 1351
Extracted: 1041
0.8078770413064361


In [183]:
predict_df[predict_df.label == False]

,_id,extracted_char,label
3,10166,白色,False
4,10166,液体,False
1,1024773,緑色,False
1,1027018,黄色,False
2,1027018,赤褐色,False
3,1027018,橙色,False
0,1052083,紫色,False
1,1055804,白色,False
0,11006,黄色,False
1,11006,液体,False


In [182]:
Counter(predict_df[predict_df.label == False].extracted_char.tolist()).most_common()

[('液体', 16),
 ('結晶', 13),
 ('信頼性', 10),
 ('黄色', 9),
 ('可能性', 9),
 ('白色', 7),
 ('安全性', 7),
 ('無色', 7),
 ('耐性', 6),
 ('反応性', 6),
 ('緑色', 4),
 ('中性', 4),
 ('紫色', 3),
 ('粉末', 3),
 ('塩基性', 3),
 ('気体', 3),
 ('青色', 3),
 ('酸性', 3),
 ('変色', 3),
 ('赤褐色', 2),
 ('橙色', 2),
 ('黒色', 2),
 ('遅効性', 2),
 ('赤色', 2),
 ('褐色', 2),
 ('刺激臭', 2),
 ('有効性', 2),
 ('灰色', 2),
 ('青緑色', 1),
 ('活性', 1),
 ('二段階変色', 1),
 ('0では黄色', 1),
 ('耐湿性', 1),
 ('少量含み緑色', 1),
 ('黄褐色', 1),
 ('耐摩耗性', 1),
 ('脂溶性', 1),
 ('熱可塑性', 1),
 ('極性', 1),
 ('不快臭', 1),
 ('弱酸性', 1),
 ('興味', 1),
 ('依存性', 1),
 ('抵抗性', 1),
 ('感受性', 1),
 ('深緑色', 1),
 ('陽性', 1),
 ('クリーム色', 1),
 ('高い無色', 1),
 ('立体異性', 1),
 ('難治性', 1),
 ('無臭', 1),
 ('不安定性', 1),
 ('物性', 1),
 ('分子性', 1),
 ('極めて反応性', 1),
 ('伝導性', 1),
 ('肝毒性', 1),
 ('求核性', 1),
 ('ため有用性', 1),
 ('親和性', 1),
 ('油性', 1),
 ('不揮発性', 1),
 ('急性', 1),
 ('黄橙色', 1),
 ('再発性抵抗性', 1),
 ('催奇形性', 1),
 ('官能性', 1),
 ('Ih対称性', 1),
 ('対称性', 1),
 ('分子対称性', 1),
 ('グルタミン酸依存性', 1),
 ('生物活性', 1),
 ('菫色', 1),
 ('オレンジ色', 1),
 ('不溶性', 1),
